In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pyt
import scipy.stats as st

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Load & read data
biz_license_data_to_load = "/content/drive/MyDrive/IJ Data Analysis/Biz Permits/Business_Licenses_-_Current_Active.csv"
building_data_to_load = "/content/drive/MyDrive/IJ Data Analysis/Biz Permits/Building_Permits.csv"
pd.set_option('display.max_rows', None)

biz_license = pd.read_csv(biz_license_data_to_load, low_memory=False)
building_permit = pd.read_csv(building_data_to_load, low_memory=False)



### Buildings Data

In [4]:
#Columns
building_permit.columns.tolist()

['ID',
 'PERMIT#',
 'PERMIT_TYPE',
 'REVIEW_TYPE',
 'APPLICATION_START_DATE',
 'ISSUE_DATE',
 'PROCESSING_TIME',
 'STREET_NUMBER',
 'STREET DIRECTION',
 'STREET_NAME',
 'SUFFIX',
 'WORK_DESCRIPTION',
 'BUILDING_FEE_PAID',
 'ZONING_FEE_PAID',
 'OTHER_FEE_PAID',
 'SUBTOTAL_PAID',
 'BUILDING_FEE_UNPAID',
 'ZONING_FEE_UNPAID',
 'OTHER_FEE_UNPAID',
 'SUBTOTAL_UNPAID',
 'BUILDING_FEE_WAIVED',
 'ZONING_FEE_WAIVED',
 'OTHER_FEE_WAIVED',
 'SUBTOTAL_WAIVED',
 'TOTAL_FEE',
 'CONTACT_1_TYPE',
 'CONTACT_1_NAME',
 'CONTACT_1_CITY',
 'CONTACT_1_STATE',
 'CONTACT_1_ZIPCODE',
 'CONTACT_2_TYPE',
 'CONTACT_2_NAME',
 'CONTACT_2_CITY',
 'CONTACT_2_STATE',
 'CONTACT_2_ZIPCODE',
 'CONTACT_3_TYPE',
 'CONTACT_3_NAME',
 'CONTACT_3_CITY',
 'CONTACT_3_STATE',
 'CONTACT_3_ZIPCODE',
 'CONTACT_4_TYPE',
 'CONTACT_4_NAME',
 'CONTACT_4_CITY',
 'CONTACT_4_STATE',
 'CONTACT_4_ZIPCODE',
 'CONTACT_5_TYPE',
 'CONTACT_5_NAME',
 'CONTACT_5_CITY',
 'CONTACT_5_STATE',
 'CONTACT_5_ZIPCODE',
 'CONTACT_6_TYPE',
 'CONTACT_6_NAME',


In [5]:
build_df = building_permit
build_df.head()

,ID,PERMIT#,PERMIT_TYPE,REVIEW_TYPE,APPLICATION_START_DATE,ISSUE_DATE,PROCESSING_TIME,STREET_NUMBER,STREET DIRECTION,STREET_NAME,...,PIN9,PIN10,COMMUNITY_AREA,CENSUS_TRACT,WARD,XCOORDINATE,YCOORDINATE,LATITUDE,LONGITUDE,LOCATION
0,1830273,100190752,PERMIT - SIGNS,SIGN PERMIT,10/16/2007,03/05/2008,141.0,6349,S,COTTAGE GROVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2060019,100296510,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,06/15/2009,06/15/2009,0.0,10000,N,BESSIE COLEMAN,...,NaN,NaN,76.0,NaN,41.0,NaN,NaN,NaN,NaN,NaN
2,1671981,100108067,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,04/17/2006,04/17/2006,0.0,7535,N,HERMITAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2832499,100696131,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,03/20/2017,03/20/2017,0.0,150,N,RIVERSIDE,...,NaN,NaN,28.0,2801.0,42.0,NaN,NaN,NaN,NaN,NaN
4,3053734,100830913,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,07/17/2019,07/17/2019,0.0,4001,N,RAVENSWOOD,...,NaN,NaN,6.0,60200.0,47.0,NaN,NaN,NaN,NaN,NaN


In [6]:
#Earliest App Start Date
# Replace NaN values with a placeholder
building_permit["APPLICATION_START_DATE"].fillna('9999-12-31', inplace=True)

# Convert to datetime format
building_permit["APPLICATION_START_DATE"] = pd.to_datetime(building_permit["APPLICATION_START_DATE"], errors='coerce')

# Find the minimum date (ignoring the placeholder date)
build_earliest_app_start_date = building_permit[building_permit["APPLICATION_START_DATE"]!='9999-12-31']["APPLICATION_START_DATE"].min()

print(build_earliest_app_start_date)


2001-07-25 00:00:00


In [7]:
#Latest App Start Date
# Find the maximum date (ignoring the placeholder date)
build_latest_app_start_date = building_permit[building_permit["APPLICATION_START_DATE"]!='9999-12-31']["APPLICATION_START_DATE"].max()

print(build_latest_app_start_date)

2023-07-26 00:00:00


In [9]:
permit_types = build_df["PERMIT_TYPE"].value_counts()
permit_types

PERMIT - ELECTRIC WIRING          260648
PERMIT - EASY PERMIT PROCESS      205153
PERMIT - RENOVATION/ALTERATION    149912
PERMIT - SIGNS                     47844
PERMIT - NEW CONSTRUCTION          27549
PERMIT - WRECKING/DEMOLITION       19971
PERMIT - ELEVATOR EQUIPMENT        18956
PERMIT - SCAFFOLDING                8861
PERMIT - REINSTATE REVOKED PMT      4175
PERMIT - PORCH CONSTRUCTION         3096
PERMIT - FOR EXTENSION OF PMT         58
Name: PERMIT_TYPE, dtype: int64

In [10]:
#REVIEW TYPES
review_types = build_df["REVIEW_TYPE"].value_counts()
review_types

EASY PERMIT WEB                   300444
EASY PERMIT                       165277
STANDARD PLAN REVIEW              127775
SIGN PERMIT                        47844
SELF CERT                          45762
DEMOLITION PERMIT                  19971
CONVEYANCE DEVICE PERMIT           18956
FIRE PROTECTION SYSTEM              7394
ELECTRICAL PLAN REVIEW              5915
TRADITIONAL DEVELOPER SERVICES      5386
DIRECT DEVELOPER SERVICES           1493
Name: REVIEW_TYPE, dtype: int64

In [12]:
#AVG COMPLETION TIME BY PERMIT TYPE

# Convert PROCESSING_TIME to numeric values (if it's not already)
build_df['PROCESSING_TIME'] = pd.to_numeric(build_df['PROCESSING_TIME'], errors='coerce')

# Group by 'PERMIT_TYPE', calculate the mean of 'PROCESSING_TIME', round it, and reset the index
avg_permit_processing_time = build_df.groupby('PERMIT_TYPE')['PROCESSING_TIME'].mean().apply(np.ceil).reset_index()

# Rename the columns and set 'PERMIT_TYPE' as index
avg_permit_processing_time.columns = ['PERMIT_TYPE', 'Avg Processing Time (Days)']
avg_permit_processing_time.set_index('PERMIT_TYPE', inplace=True)

# Sort the DataFrame by 'Avg Processing Time (Days)' in descending order
avg_permit_processing_time = avg_permit_processing_time.sort_values(by='Avg Processing Time (Days)', ascending=False)

avg_permit_processing_time

,Avg Processing Time (Days)
PERMIT_TYPE,
PERMIT - SIGNS,80.0
PERMIT - NEW CONSTRUCTION,41.0
PERMIT - WRECKING/DEMOLITION,36.0
PERMIT - RENOVATION/ALTERATION,35.0
PERMIT - ELECTRIC WIRING,22.0
PERMIT - PORCH CONSTRUCTION,21.0
PERMIT - FOR EXTENSION OF PMT,3.0
PERMIT - REINSTATE REVOKED PMT,3.0
PERMIT - EASY PERMIT PROCESS,2.0


In [13]:
# AVG COMPLETION TIME BY REVIEW TYPE

# Convert PROCESSING_TIME to numeric values (if it's not already)
build_df['PROCESSING_TIME'] = pd.to_numeric(build_df['PROCESSING_TIME'], errors='coerce')

# Group by 'REVIEW_TYPE', calculate the mean of 'PROCESSING_TIME', round it, and reset the index
avg_review_processing_time = build_df.groupby('REVIEW_TYPE')['PROCESSING_TIME'].mean().apply(np.ceil).reset_index()

# Rename the columns and set 'REVIEW_TYPE' as index
avg_review_processing_time.columns = ['REVIEW_TYPE', 'Avg Processing Time (Days)']
avg_review_processing_time.set_index('REVIEW_TYPE', inplace=True)

# Sort the DataFrame by 'Avg Processing Time (Days)' in descending order
avg_review_processing_time = avg_review_processing_time.sort_values(by='Avg Processing Time (Days)', ascending=False)

avg_review_processing_time


,Avg Processing Time (Days)
REVIEW_TYPE,
SIGN PERMIT,80.0
DIRECT DEVELOPER SERVICES,50.0
STANDARD PLAN REVIEW,47.0
DEMOLITION PERMIT,36.0
ELECTRICAL PLAN REVIEW,35.0
FIRE PROTECTION SYSTEM,23.0
EASY PERMIT WEB,18.0
TRADITIONAL DEVELOPER SERVICES,10.0
SELF CERT,6.0


### BACP Data

In [14]:
#Columns
biz_license.columns.tolist()

['ID',
 'LICENSE ID',
 'ACCOUNT NUMBER',
 'SITE NUMBER',
 'LEGAL NAME',
 'DOING BUSINESS AS NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP CODE',
 'WARD',
 'PRECINCT',
 'WARD PRECINCT',
 'POLICE DISTRICT',
 'LICENSE CODE',
 'LICENSE DESCRIPTION',
 'BUSINESS ACTIVITY ID',
 'BUSINESS ACTIVITY',
 'LICENSE NUMBER',
 'APPLICATION TYPE',
 'APPLICATION CREATED DATE',
 'APPLICATION REQUIREMENTS COMPLETE',
 'PAYMENT DATE',
 'CONDITIONAL APPROVAL',
 'LICENSE TERM START DATE',
 'LICENSE TERM EXPIRATION DATE',
 'LICENSE APPROVED FOR ISSUANCE',
 'DATE ISSUED',
 'LICENSE STATUS',
 'LICENSE STATUS CHANGE DATE',
 'SSA',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [15]:
biz_df = biz_license
biz_df.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,2918629-20230713,2918629,35751,211,"CHICAGO RUNNING & SPECIAL EVENTS MANAGEMENT, INC.","CHICAGO RUNNING & SPECIAL EVENTS MANAGEMENT, INC.",2221 W 43RD ST,CHICAGO,IL,60609,...,07/13/2023,07/29/2023,07/13/2023,07/13/2023,AAI,NaN,NaN,41.815621,-87.680636,"(41.81562105420659, -87.68063583315673)"
1,2923792-20230720,2923792,499455,1,YOUR PASSION 1ST,YOUR PASSION 1ST,120 LAKE ST.,OAK PARK,IL,60302,...,07/20/2023,07/29/2023,07/20/2023,07/20/2023,AAI,NaN,NaN,NaN,NaN,NaN
2,2924066-20230726,2924066,222168,6,FOGO DE CHAO CHURRASCARIA (CHICAGO) LLC,FOGO DE CHAO,661 N LA SALLE DR,CHICAGO,IL,60654,...,07/26/2023,07/29/2023,07/26/2023,07/26/2023,AAI,NaN,NaN,41.894125,-87.632516,"(41.8941247035236, -87.63251632012691)"
3,2918453-20230710,2918453,499107,1,CELIA ECHT,CEAL'S ROSE,2924 N CENTRAL PARK AVE,CHICAGO,IL,60618,...,07/10/2023,07/30/2023,07/10/2023,07/10/2023,AAI,NaN,NaN,41.934553,-87.717407,"(41.93455317548739, -87.71740714565092)"
4,2918507-20230712,2918507,317904,126,"CITY MARKETS, INC.","CITY MARKETS, INC.",47 W DIVISION ST 1 187,CHICAGO,IL,60610,...,07/12/2023,07/30/2023,07/12/2023,07/12/2023,AAI,NaN,NaN,41.903809,-87.630407,"(41.90380882550438, -87.63040710682779)"


In [16]:
#Earliest App Start Date
# Replace NaN values with a placeholder
biz_df["APPLICATION CREATED DATE"].fillna('9999-12-31', inplace=True)

# Convert to datetime format
biz_df["APPLICATION CREATED DATE"] = pd.to_datetime(biz_df["APPLICATION CREATED DATE"], errors='coerce')

# Find the minimum date (ignoring the placeholder date)
biz_earliest_app_start_date = biz_df[biz_df["APPLICATION CREATED DATE"]!='9999-12-31']["APPLICATION CREATED DATE"].min()

print(biz_earliest_app_start_date)

2008-04-11 00:00:00


In [17]:
biz_latest_app_start_date = biz_df[biz_df["APPLICATION CREATED DATE"]!='9999-12-31']["APPLICATION CREATED DATE"].max()

print(biz_latest_app_start_date)

2023-07-27 00:00:00


In [18]:
# Convert 'APPLICATION CREATED DATE' and 'DATE ISSUED' to datetime
biz_df['APPLICATION CREATED DATE'] = pd.to_datetime(biz_df['APPLICATION CREATED DATE'])
biz_df['DATE ISSUED'] = pd.to_datetime(biz_df['DATE ISSUED'])

# Create a new column 'PROCESSING_TIME' that is the difference in days between 'APPLICATION CREATED DATE' and 'DATE ISSUED'
biz_df['PROCESSING_TIME'] = (biz_df['DATE ISSUED'] - biz_df['APPLICATION CREATED DATE']).dt.days

# Group by 'LICENSE DESCRIPTION', calculate the mean of 'PROCESSING_TIME', round it, and reset the index
avg_license_processing_time = biz_df.groupby('LICENSE DESCRIPTION')['PROCESSING_TIME'].mean().apply(np.ceil).reset_index()

# Rename the columns and set 'LICENSE DESCRIPTION' as index
avg_license_processing_time.columns = ['LICENSE DESCRIPTION', 'Avg Processing Time (Days)']
avg_license_processing_time.set_index('LICENSE DESCRIPTION', inplace=True)

# Sort the DataFrame by 'Avg Processing Time (Days)' in descending order
avg_license_processing_time = avg_license_processing_time.sort_values(by='Avg Processing Time (Days)', ascending=False)

avg_license_processing_time


,Avg Processing Time (Days)
LICENSE DESCRIPTION,
Late Hour,684.0
Tavern,398.0
Outdoor Patio,249.0
Outdoor Entertainment Venue,224.0
Public Place of Amusement,205.0
Food - Shared Kitchen,175.0
Consumption on Premises - Incidental Activity,171.0
Mobile Food License,167.0
Food - Shared Kitchen - Supplemental,153.0


### Zoning Data